In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
import typing as T
import re
import collections

NUM = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
MINUS = r'(?P<MINUS>-)'
TIMES = r'(?P<TIMES>\*)'
DIVIDE = r'(?P<DIVIDE>/)'
LPAREN = r'(?P<LPAREN>\()'
RPAREN = r'(?P<RPAREN>\))'
WS = r'(?P<WS>\s+)'

master_pat = re.compile('|'.join([
    NUM, PLUS, MINUS, TIMES, DIVIDE, LPAREN, RPAREN, WS
]))


class Token(T.NamedTuple):
    type: str
    value: str


def generate_tokens(text: str) -> T.Iterable[Token]:
    scanner: re.Scanner = master_pat.scanner(text)
    for m in iter(scanner.match, None):  # type: re.Match
        token = Token(
            type=str(m.lastgroup),
            value=m.group()
        )
        if token.type != 'WS':
            yield token


class Evaluator:

    def parse(self, text: str) -> T.Any:
        self.tokens: T.Iterable[Token] = generate_tokens(text)
        self.last_token = None
        self.next_token = None
        self._advance()
        return self.expr()

    def _advance(self):
        self.last_token, self.next_token = self.next_token, next(self.tokens, None)  # noqa: E501

    def _accept(self, token_type: str) -> bool:
        if self.next_token and self.next_token.type == token_type:
            self._advance()
            return True
        else:
            return False

    def _expect(self, token_type: str):
        if not self._accept(token_type):
            raise SyntaxError(f'expected: {token_type}')

    def expr(self):
        # expr ::= term { ('+'\'-') term }*

        expr_value = self.term()
        while self._accept('PLUS') or self._accept('MINUS'):
            op = self.last_token.type
            right = self.term()
            if op == 'PLUS':
                expr_value += right
            elif op == 'MINUS':
                expr_value -= right

        return expr_value

    def term(self):
        # term ::= factor { ('*'|'/') factor }*

        term_value = self.factor()
        while self._accept('TIMES') or self._accept('DIVIDE'):
            op = self.last_token.type
            right = self.factor()
            if op == 'TIMES':
                term_value *= right
            elif op == 'DIVIDE':
                term_value /= right

        return term_value

    def factor(self):
        # factor ::= NUM | ( expr )

        if self._accept('NUM'):
            return float(self.last_token.value)
        elif self._accept('LPAREN'):
            expr_value = self.expr()
            self._expect('RPAREN')
            return expr_value
        else:
            raise SyntaxError('expect: NUMBER or LPAREN')


class TreeBuilder(Evaluator):

    def expr(self):
        # expr ::= term { ('+'\'-') term }*

        expr_value = self.term()
        while self._accept('PLUS') or self._accept('MINUS'):
            op = self.last_token.type
            right = self.term()
            if op == 'PLUS':
                expr_value = ('+', expr_value, right)
            if op == 'MINUS':
                expr_value = ('-', expr_value, right)

        return expr_value

    def term(self):
        # term ::= factor { ('*'|'/') factor }*

        term_value = self.factor()
        while self._accept('TIMES') or self._accept('DIVIDE'):
            op = self.last_token.type
            right = self.factor()
            if op == 'TIMES':
                term_value = ('*', term_value, right)
            elif op == 'DIVIDE':
                term_value = ('/', term_value, right)

        return term_value

    def factor(self):
        # factor ::= NUM | ( expr )

        if self._accept('NUM'):
            return float(self.last_token.value)
        elif self._accept('LPAREN'):
            expr_value = self.expr()
            self._expect('RPAREN')
            return expr_value
        else:
            raise SyntaxError('expect: NUMBER or LPAREN')


"""

>>> e = Evaluator()
>>> e.parse('2')
2.0
>>> e.parse('2 + 3')
5.0
>>> e.parse('2 + 3 * 4')
14.0
>>> e.parse('2 + (3 + 4) * 5')
37.0
>>> e.parse('2 + (3 + * 4)')
Traceback (most recent call last):
    ...
SyntaxError: expect: NUMBER or LPAREN

>>> b = TreeBuilder()
>>> b.parse('2')
2.0
>>> b.parse('2 + 3')
('+', 2.0, 3.0)
>>> b.parse('2 + 3 * 4')
('+', 2.0, ('*', 3.0, 4.0))
>>> b.parse('2 + (3 + 4) * 5')
('+', 2.0, ('*', ('+', 3.0, 4.0), 5.0))
>>> b.parse('2 + (3 + * 4)')
Traceback (most recent call last):
    ...
SyntaxError: expect: NUMBER or LPAREN
"""

%mypy
doctest.testmod()

Success: no issues found in 1 source file
Type checking successful


TestResults(failed=0, attempted=12)

In [4]:
!pip install ply

from ply.lex import lex
from ply.yacc import yacc

tokens = ['NUM', 'PLUS', 'MINUS', 'TIMES', 'DIVIDE', 'LPAREN', 'RPAREN']
t_ignore = '\t\n'

t_PLUS = r'\+'
t_MINUS = r'-'
t_TIMES = r'\*'
t_DIVIDE = r'/'
t_LPAREN = r'\('
t_RPAREN = r'\)'


def t_NUM(t):
    r'\d+'
    t.value = int(t.value)
    return t


def t_error(t):
    print('bad character: {!r}'.format(t.value[0]))
    t.skip(1)


lexer = lex()

def p_expr(p):
    """
    expr : expr PLUS term
         | expr MINUS term
    """
    if p[2] == '+':
        p[0] = p[1] + p[3]
    elif p[2] == '-':
        p[0] = p[1] - p[3]


def p_expr_term(p):
    """
    expr : term
    """
    p[0] = p[1]


def p_term(p):
    """
    term : term TIMES factor
         | term DIVIDE factor
    """
    if p[2] == '*':
        p[0] = p[1] * p[3]
    elif p[2] == '/':
        p[0] = p[1] / p[3]


def p_term_factor(p):
    """
    term : factor
    """
    p[0] = p[1]


def p_factor(p):
    """
    factor : NUM
    """
    p[0] = p[1]


def p_factor_group(p):
    """
    factor : LPAREN expr RPAREN
    """
    p[0] = p[2]


def p_error(p):
    print('Syntax error')


"""
>>> parser = yacc()
>>> parser.parse('2')
2
>>> parser.parse('2+3')
5
>>> parser.parse('2+(3+4)*5')
37
"""

doctest.testmod()

TypeError: <module '__main__'> is a built-in module